In [1]:
import numpy as np
import torch
import torchvision
import torch.optim as optim
from datagen import *
from datasets import *
from modules import *
from nets import *
from backdoor import Backdoor

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
backdoor = Backdoor(ConvNet, config='11-layer', channels=1, classes=10)
backdoor.load_data(torchvision.datasets.MNIST, etype=ImageEntity)
backdoor.create_models(optim.SGD, torch.nn.CrossEntropyLoss, lr=0.01)


In [3]:
patches = ImagePatch((9, 9), 1, 'random').get_patches(len(backdoor.traindata))
targets = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 0}
operations = ([ExpandTransform(1)], [RotateTransform()], [LabelTransform(targets)])
merge = OverlayMerge(select=True)

In [4]:
backdoor.poison(patches, operations, merge, 0.2)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
backdoor.train('base', epochs=1, verbose=True)

Training started
Epoch 1/1 | 24.96% | Loss: 0.0429 | Samples trained: 14976/60000
Epoch 1/1 | 49.92% | Loss: 0.0362 | Samples trained: 29952/60000
Epoch 1/1 | 74.88% | Loss: 0.0354 | Samples trained: 44928/60000
Epoch 1/1 | 99.84% | Loss: 0.0331 | Samples trained: 59904/60000
Epoch 1 complete | Loss: 0.0333
Training complete | Net Average Loss: 0.0333 | Total epochs: 1


(0.033292236821801637, None)

In [9]:
backdoor.test('base', True, device)

Testing started
24.92% Testing complete | Loss: 0.0513 | Accuracy: 0.9852
49.84% Testing complete | Loss: 0.0521 | Accuracy: 0.9848
74.76% Testing complete | Loss: 0.0408 | Accuracy: 0.9877
99.68% Testing complete | Loss: 0.0358 | Accuracy: 0.9892
Testing complete | Loss: 0.0357 | Accuracy: 0.9892


(0.035700748725741485, 0.9892, None, None)

In [7]:
backdoor.eval(device=device)

Accuracy on Clean | Base 98.79% | Trojan 98.33% | Difference 0.46%
Base Accuracy on Poison 1.48% | Attack Success Rate (ASR): 91.96%


(0.9879, 0.9833, 0.0148, 0.9344)